In [1]:
import bs4
import pandas
    
from urllib import request

In [2]:
list_question_title = []

In [3]:
for i in range (1):
    url = "https://stackoverflow.com/questions?tab=newest&page="+str(i)
    request_text = request.urlopen(url).read()
    page = bs4.BeautifulSoup(request_text,'html.parser')


In [4]:
for item2 in page.find_all('a',{'class':'s-link'}):
       list_question_title.append(item2.get_text())

In [5]:
print(list_question_title)

['\n\n', '\n\n', 'Splitting Date Range into Months in Excel', "PHP | Google Translate API doesn't translate FR text", 'How to debug in Visual Studio C++ without PDB', 'How do I count the number of paths in this problem?', "how to cancel mongoose query 'save' 'pre' hook", 'Spring Session on Spring Boot using MapSessionRepositiry as session storage on non-clustered persistence', 'Is it possible to access S2S VPN via Client VPN?', "No module named 'core' django app deployed in Railway", 'Unity: audio gets stuck when animation is played', 'get age in years for month of july', 'Bot status Discord.js 14.6.0', 'WooCommerce: Set shipping method to free by selected payment method', 'How to use firebase with flutter', 'new Date() render the wrong date format', 'VS code Wrong C output', 'How can I make my python program choose option by itself randomly if I kept it in one variable?', 'How to use Serilog with Windows Client (net6.0-windows) application', 'Django Verbatum like template tag to show 